In [59]:
from pattern.web import Wikipedia, plaintext
#from string import punctuation

import re
from string import digits, punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams
from collections import Counter

Вам необходимо написать программу, которая парсит статьи википедии (language=en) и считает по ним некоторые статистики.

1. Реализовать класс WikiParser с конструктором без аргументов и методом get_articles, который принимает название исходной статьи start, обходит все статьи википедии, на которые она ссылается, и возвращает список содержимого статей (с выполнением парсинга).

    При парсинге каждой статьи для того, чтобы убрать html теги, используется функция pattern.web.plaintext.
    В результате парсинга между соседними словами должно быть 1 пробел.
    В результате парсинга весь текст переводится в lowercase, знаки пунктуации выкидываются


In [63]:
 # task 1
class WikiParser:
    def __init__(self):
        pass
    
    def clean_text(self, text):
        text = text.lower()
        text = text.translate(str.maketrans('', '', digits))
        text = text.translate(str.maketrans('', '', punctuation)) #не удаляет дефисы, но думаю, это уместно
        clean_text = re.sub('\s{2,}', ' ', text)
        return clean_text
    
    def get_articles(self, start):
        wiki = Wikipedia(language='en')
        start = wiki.article(start)
        list_of_strings = list()
        for link in start.links:
            article = wiki.article(link)
            if article is not None:
                article = plaintext(article.source)
                list_of_strings.append(self.clean_text(article))
        return list_of_strings

Реализовать класс TextStatistics с конструктором, который принимает в качестве аргумента список содержимого статей:

    get_top_3grams - возвращает tuple, первым элемент которого - список 3-грамм в порядке убывания их частот, второй элемент - соотвественно список сколько раз встретилась каждая 3грамма. Подсчет идет по всему корпусу articles. При подсчете 3-грамм исключить из рассмотрения все числа и пунктуацию.
    get_top_words - возвращает tuple, первым элемент которого - список слов в порядке убывания их частот, второй элемент - соотвественно список сколько раз встретилась каждое слово. Подсчет идет по всему корпусу articles. При подсчете слов исключить из рассмотрения все числа, предлоги, артикли и пунктуацию.


In [96]:
 # task 2
class TextStatistics:
    def __init___(self):
        pass
    
    def get_top_3grams(self, articles, n):
        all_3grams = list()
        for article in articles:
            for ngram in ngrams(word_tokenize(article), 3):
                all_3grams.append(ngram)
        all_3grams = Counter(all_3grams)
        ngram_list = list()
        freq_list = list()
        for ngram, freq in all_3grams.most_common()[:n]:
            ngram_list.append(ngram)
            freq_list.append(freq)
        return (ngram_list, freq_list)
    
    def get_top_words(self, articles, n):
        words = list()
        for article in articles:
            stop_words = set(stopwords.words('english'))
            tokenizer = RegexpTokenizer(r'\w+')
            tokens = tokenizer.tokenize(article)
            for token in tokens:
                if token not in stop_words:
                    words.append(token)
        words = Counter(words)
        top_words = words.most_common()[:n]
        word_list = list()
        freq_list = list()
        for word, freq in top_words:
            word_list.append(word)
            freq_list.append(freq)
        return (word_list, freq_list)

Реализовать класс Experiment с методом show_results, который используя WikiParser и TextStatistics:

    Выполняет парсинг статей википедии для "Natural language processing"
    По полученному корпусу текстов считает топ-20 3-грамм и топ-20 слов.
    По статье "Natural language processing" (только по ней) считает топ-5 3-грамм и топ-5 слов.
    Печатает результаты эксперимента в структурированной форме


In [146]:
# task 3
class Experiment:
    def __init__(self):
        pass
            
    def show_results(self):
        corpus = WikiParser().get_articles('Natural language processing')
        top_words = TextStatistics().get_top_words(corpus, 20)
        print('Top-20 words of the whole corpora' + '\n')
        for x, y in zip(top_words[0],top_words[1]):
            print(x + ' - ' + str(y) + '\n')
            
        top_3grams = TextStatistics().get_top_3grams(corpus, 20)
        print('\n' + 'Top-20 3grams of the whole corpora' + '\n')
        for x, y in zip(top_3grams[0],top_3grams[1]):
            print(str(x) + ' - ' + str(y) + '\n')
        
        nlp = plaintext(wiki.article('Natural language processing').source)
        nlp = [WikiParser().clean_text(nlp)]
        
        top_5_words = TextStatistics().get_top_words(nlp, 5)
        print('\n' + 'Top-5 words of \'Natural language processing\'' + '\n')
        for x, y in zip(top_5_words[0],top_5_words[1]):
            print(x + ' - ' + str(y) + '\n')
            
        top_5_3grams = TextStatistics().get_top_3grams(nlp, 5)
        print('\n' + 'Top-5 3grams of \'Natural language processing\'' + '\n')
        for x, y in zip(top_5_3grams[0],top_5_3grams[1]):
            print(str(x) + ' - ' + str(y) + '\n')

In [148]:
experiment = Experiment().show_results()

Top-20 words of the whole corpora

language - 3970

english - 1836

speech - 1754

retrieved - 1697

languages - 1687

words - 1649

also - 1643

word - 1565

used - 1534

text - 1487

p - 1411

may - 1309

displaystyle - 1241

one - 1240

machine - 1227

learning - 1188

example - 1129

isbn - 1122

use - 1068

b - 1041


Top-20 3grams of the whole corpora

('from', 'the', 'original') - 378

('archived', 'from', 'the') - 366

('natural', 'language', 'processing') - 329

('the', 'original', 'on') - 308

('v', 't', 'e') - 251

('the', 'use', 'of') - 228

('as', 'well', 'as') - 210

('one', 'of', 'the') - 210

('proceedings', 'of', 'the') - 187

('a', 'b', 'c') - 183

('the', 'european', 'union') - 164

('cambridge', 'university', 'press') - 153

('such', 'as', 'the') - 151

('of', 'the', 'european') - 151

('the', 'number', 'of') - 142

('for', 'example', 'the') - 137

('university', 'press', 'isbn') - 131

('a', 'number', 'of') - 129

('a', 'set', 'of') - 125

('based', 'on', 'the') - 